**World Development Indicators (WDI) Data**

World Development Indicators (WDI) by World Bank includes data spanning up to 56 years—from 1960 to 2016. WDI frames global trends with indicators on population, population density, urbanization, GNI, and GDP. These indicators measure the world’s economy and progress toward improving lives, achieving sustainable development, providing support for vulnerable populations, and reducing gender disparities.

**Indonesia GDP Builder**

In this notebook we will focus more in components that make up a Indonesia’s GDP, especially the foreign balance of trade.

In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
dataset_ref = client.dataset('worldbank_wdi', project='patents-public-data')
dataset = client.get_dataset(dataset_ref)

In [5]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

wdi_2016


In [6]:
table_ref = dataset_ref.table('wdi_2016')
table=client.get_table(table_ref)

In [7]:
table.schema

[SchemaField('year', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('country_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('country_code', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('indicator_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('indicator_code', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('indicator_value', 'FLOAT', 'NULLABLE', None, (), None)]

**CONTENT IN THIS DATASET**

In [8]:
client.list_rows(table, max_results=10).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,year,country_name,country_code,indicator_name,indicator_code,indicator_value
0,1960,Chad,TCD,Discrepancy in expenditure estimate of GDP (cu...,NY.GDP.DISC.CN,6.990439e+09
1,1961,Chad,TCD,"Death rate, crude (per 1,000 people)",SP.DYN.CDRT.IN,2.624000e+01
2,1961,Chad,TCD,GNI per capita (constant LCU),NY.GNP.PCAP.KN,2.846799e+05
3,1961,Chad,TCD,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R4.ZS,9.389671e-01
4,1961,Chad,TCD,"Merchandise imports by the reporting economy, ...",TM.VAL.MRCH.RS.ZS,9.243028e+01
5,1962,Chad,TCD,GDP per capita (current US$),NY.GDP.PCAP.CD,1.145403e+02
6,1962,Chad,TCD,GNI growth (annual %),NY.GNP.MKTP.KD.ZG,5.245336e+00
7,1962,Chad,TCD,Gross capital formation (% of GDP),NE.GDI.TOTL.ZS,1.167231e+01
8,1962,Chad,TCD,Gross capital formation (current US$),NE.GDI.TOTL.CD,4.174435e+07
9,1962,Chad,TCD,Household final consumption expenditure (curre...,NE.CON.PRVT.CN,6.332295e+10


**How Many Years, Countires, and Indicators in This Dataset**

In [9]:
query = """
        SELECT COUNT(DISTINCT(year)) AS num_year,
               COUNT(DISTINCT(country_name)) AS num_countires,
               COUNT(DISTINCT(indicator_name)) AS num_indicator
        FROM `patents-public-data.worldbank_wdi.wdi_2016`
        """
result = client.query(query).to_dataframe()
result.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,num_year,num_countires,num_indicator
0,57,264,1446


**WHEN DATA STARTS AND ENDS?**

In [10]:
query = """
        SELECT MIN(year) AS start_record,
               MAX(year) AS end_record
        FROM `patents-public-data.worldbank_wdi.wdi_2016`
        """
result = client.query(query).to_dataframe()
result.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,start_record,end_record
0,1960,2016


**What Components That Add Value to Indonesia's GDP**

In [11]:
query = """
        SELECT country_name,indicator_name, indicator_code
        FROM `patents-public-data.worldbank_wdi.wdi_2016`
        WHERE country_name = 'Indonesia' AND indicator_name LIKE '%value added (% of GDP)%' AND year = 2006 
        """
result = client.query(query).to_dataframe()
result.head(50)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,country_name,indicator_name,indicator_code
0,Indonesia,"Agriculture, value added (% of GDP)",NV.AGR.TOTL.ZS
1,Indonesia,"Manufacturing, value added (% of GDP)",NV.IND.MANF.ZS
2,Indonesia,"Services, etc., value added (% of GDP)",NV.SRV.TETC.ZS
3,Indonesia,"Industry, value added (% of GDP)",NV.IND.TOTL.ZS


**From 2006 to 2015 (10 Years) what percentage was added to GDP by each component**

In [12]:
query = """
        SELECT year, agriculture, services, industry, manufacturing
        FROM (SELECT year,
              SUM(IF(indicator_code = 'NV.AGR.TOTL.ZS' AND country_name = 'Indonesia', indicator_value, 0)) agriculture,
              SUM(IF(indicator_code = 'NV.SRV.TETC.ZS' AND country_name = 'Indonesia', indicator_value, 0)) services,
              SUM(IF(indicator_code = 'NV.IND.TOTL.ZS' AND country_name = 'Indonesia', indicator_value, 0)) industry,
              SUM(IF(indicator_code = 'NV.IND.MANF.ZS' AND country_name = 'Indonesia', indicator_value, 0)) manufacturing
              FROM `patents-public-data.worldbank_wdi.wdi_2016`
              GROUP BY year
              )
        WHERE year >= 2006 AND year < 2016 
        """
result = client.query(query).to_dataframe()
result.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,agriculture,services,industry,manufacturing
0,2006,12.973803,40.082639,46.943559,27.537574
1,2007,13.716683,39.484175,46.799142,27.048413
2,2008,14.481740,37.457515,48.060745,27.814273
3,2009,15.290148,37.057730,47.652123,26.355475
4,2010,13.929213,40.665990,42.775866,22.038629
5,2011,13.512287,40.584604,43.913423,21.760855
6,2012,13.373974,40.873375,43.593744,21.450955
7,2013,13.356699,41.516537,42.635677,21.028689
8,2014,13.341663,42.267359,41.897344,21.005866
9,2015,13.520732,43.323845,40.014807,20.842671


Of the four components, only services whose value increases every year the rest has decreased.

**Agricultural Land Area**

In [13]:
query = """
        SELECT year, indicator_name, indicator_value
        FROM `patents-public-data.worldbank_wdi.wdi_2016`
        WHERE country_name = 'Indonesia' AND indicator_code = 'AG.LND.ARBL.HA' AND year >= 2006 AND year <= 2016
        ORDER BY year 
        """
result = client.query(query).to_dataframe()
result.head(11)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,indicator_name,indicator_value
0,2006,Arable land (hectares),21500000.0
1,2007,Arable land (hectares),22000000.0
2,2008,Arable land (hectares),22700000.0
3,2009,Arable land (hectares),23600000.0
4,2010,Arable land (hectares),23600000.0
5,2011,Arable land (hectares),23500000.0
6,2012,Arable land (hectares),23500000.0
7,2013,Arable land (hectares),23500000.0
8,2014,Arable land (hectares),0.0
9,2015,Arable land (hectares),0.0


Although Indonesia's income from agriculture has not yet been able to touch the highest gain in 2009, this is in line with the decrease in available land, which in 2009 amounted to 23,600,000 hectares but in 2011 decreased by 100,000 hectares to 23,500,000 hectares

In [14]:
pd.options.display.max_colwidth = 150

**Indonesia's Merchandise Export Performance**

In [15]:
query = """
        SELECT indicator_name, perc_06, perc_14, 
                perc_14 - perc_06 AS change
        FROM (
            SELECT indicator_name,
                   SUM(IF(year=2006 AND country_name = 'Indonesia', indicator_value, 0)) perc_06,
                   SUM(IF(year=2014 AND country_name = 'Indonesia', indicator_value, 0)) perc_14
            FROM `patents-public-data.worldbank_wdi.wdi_2016`
            GROUP BY indicator_name
        )
        WHERE indicator_name LIKE '%Merchandise exports to%' 
        ORDER BY perc_14 DESC
        """
query_job = client.query(query).to_dataframe()
query_job.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,indicator_name,perc_06,perc_14,change
0,Merchandise exports to high-income economies (% of total merchandise exports),72.168842,60.502151,-11.666691
1,Merchandise exports to low- and middle-income economies within region (% of total merchandise exports),17.837694,23.244176,5.406482
2,Merchandise exports to low- and middle-income economies in East Asia & Pacific (% of total merchandise exports),17.837694,23.244176,5.406482
3,Merchandise exports to low- and middle-income economies outside region (% of total merchandise exports),9.922806,16.225811,6.303005
4,Merchandise exports to low- and middle-income economies in South Asia (% of total merchandise exports),4.973530,9.180459,4.206930
5,Merchandise exports to economies in the Arab World (% of total merchandise exports),3.213765,4.524214,1.310449
6,Merchandise exports to low- and middle-income economies in Sub-Saharan Africa (% of total merchandise exports),1.235846,2.371431,1.135584
7,Merchandise exports to low- and middle-income economies in Latin America & the Caribbean (% of total merchandise exports),1.260900,1.801853,0.540953
8,Merchandise exports to low- and middle-income economies in Middle East & North Africa (% of total merchandise exports),1.434256,1.665164,0.230909
9,Merchandise exports to low- and middle-income economies in Europe & Central Asia (% of total merchandise exports),1.018274,1.206903,0.188629


Indonesia's merchandise export performance to high-income economies for nine years has decreased by 11 percent, in contrast to exports to other countries, which have increased.

In [16]:
query = """
        SELECT year, aggriculture_raw, manufacture, fuel, food, ores_metal
        FROM(
            SELECT year,
                SUM(IF(indicator_code = 'TX.VAL.AGRI.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) aggriculture_raw,
                SUM(IF(indicator_code = 'TX.VAL.MANF.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) manufacture,
                SUM(IF(indicator_code = 'TX.VAL.FUEL.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) fuel,
                SUM(IF(indicator_code = 'TX.VAL.FOOD.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) food,
                SUM(IF(indicator_code = 'TX.VAL.MMTL.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) ores_metal
            FROM `patents-public-data.worldbank_wdi.wdi_2016`
            GROUP BY year
        )
        WHERE year >= 2006 AND year < 2016 
        """
result = client.query(query).to_dataframe()
result.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,aggriculture_raw,manufacture,fuel,food,ores_metal
0,2006,6.414182,44.687300,27.243644,11.682106,9.968306
1,2007,6.254994,42.929868,25.386733,14.706623,10.718653
2,2008,6.433800,38.812444,29.100009,17.686060,7.963389
3,2009,4.526483,40.580969,28.420484,17.299637,9.169703
4,2010,6.539272,37.497790,29.726910,16.364015,9.866412
5,2011,7.520474,34.160795,34.141076,16.327164,7.847011
6,2012,5.940110,36.176895,33.590891,17.916767,6.342808
7,2013,5.799762,37.800947,31.623960,17.669870,7.102426
8,2014,4.827440,40.935421,29.179762,20.277902,4.776750
9,2015,0.000000,0.000000,0.000000,0.000000,0.000000


Indonesia has five merchandise exports components: agriculture, manufacture, fuel, food, ores and metal. Only food components have improved performance yearly, and manufacture components contribute the highest value.

**Indonesia's Merchandise Import Performance**

In [17]:
query = """
        SELECT indicator_name, perc_06, perc_14, 
                perc_14 - perc_06 AS change
        FROM (
            SELECT indicator_name,
                   SUM(IF(year=2006 AND country_name = 'Indonesia', indicator_value, 0)) perc_06,
                   SUM(IF(year=2014 AND country_name = 'Indonesia', indicator_value, 0)) perc_14
            FROM `patents-public-data.worldbank_wdi.wdi_2016`
            GROUP BY indicator_name
        )
        WHERE indicator_name LIKE '%Merchandise imports from%' 
        ORDER BY perc_14 DESC
        """
query_job = client.query(query).to_dataframe()
query_job.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,indicator_name,perc_06,perc_14,change
0,Merchandise imports from high-income economies (% of total merchandise imports),69.519144,59.109778,-10.409367
1,Merchandise imports from low- and middle-income economies within region (% of total merchandise imports),23.184611,31.238273,8.053661
2,Merchandise imports from low- and middle-income economies in East Asia & Pacific (% of total merchandise imports),23.184611,31.238273,8.053661
3,Merchandise imports from low- and middle-income economies outside region (% of total merchandise imports),6.587182,9.527200,2.940018
4,Merchandise imports from economies in the Arab World (% of total merchandise imports),9.974496,7.018015,-2.956481
5,Merchandise imports from low- and middle-income economies in Sub-Saharan Africa (% of total merchandise imports),0.816904,2.688304,1.871400
6,Merchandise imports from low- and middle-income economies in South Asia (% of total merchandise imports),2.471562,2.373486,-0.098076
7,Merchandise imports from low- and middle-income economies in Europe & Central Asia (% of total merchandise imports),0.893774,2.306564,1.412790
8,Merchandise imports from low- and middle-income economies in Latin America & the Caribbean (% of total merchandise imports),1.093657,1.671295,0.577637
9,Merchandise imports from low- and middle-income economies in Middle East & North Africa (% of total merchandise imports),1.311285,0.487551,-0.823734


It seems that Indonesia is starting to reduce merchandise imports from countries with high economies, Africa, and countries in the Arab world.

In [18]:
query = """
        SELECT year, aggriculture_raw, manufacture, fuel, food, ores_metal
        FROM( SELECT year,
            SUM(IF(indicator_code = 'TM.VAL.AGRI.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) aggriculture_raw,
            SUM(IF(indicator_code = 'TM.VAL.MANF.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) manufacture,
            SUM(IF(indicator_code = 'TM.VAL.FUEL.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) fuel,
            SUM(IF(indicator_code = 'TM.VAL.FOOD.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) food,
            SUM(IF(indicator_code = 'TM.VAL.MMTL.ZS.UN' AND country_name = 'Indonesia', indicator_value, 0)) ores_metal
        FROM `patents-public-data.worldbank_wdi.wdi_2016`
        GROUP BY year
        )
        WHERE year >= 2006 AND year < 2016
        """
result = client.query(query).to_dataframe()
result.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,aggriculture_raw,manufacture,fuel,food,ores_metal
0,2006,3.412065,52.659352,31.511200,8.858846,3.544132
1,2007,3.512012,52.591059,29.703376,10.550151,3.571225
2,2008,3.039945,61.890512,23.850582,7.257985,3.942071
3,2009,2.801006,65.329671,19.805166,8.916692,3.063062
4,2010,3.062639,63.319591,20.393052,8.459413,3.418741
5,2011,3.186347,59.741765,23.114475,9.432430,3.456816
6,2012,2.622239,62.435483,22.371880,8.252406,3.130052
7,2013,2.719428,59.965542,24.446542,8.779469,3.141439
8,2014,2.894254,58.854158,24.718463,9.552648,3.174817
9,2015,0.000000,0.000000,0.000000,0.000000,0.000000


It seems that Indonesia's imports of manufacture merchandise are higher than their export value, in contrast to agriculture, food, fuel, ores and metal.

**Indonesia's Balance Of Trade**

In [19]:
query = """
        SELECT year, merch_export_us_dollar, merch_import_us_dollar,
        merch_export_us_dollar - merch_import_us_dollar AS difference
        FROM ( SELECT year,
               SUM(IF(indicator_code = 'TM.VAL.MRCH.CD.WT' AND country_name = 'Indonesia', indicator_value, 0)) merch_export_us_dollar,
               SUM(IF(indicator_code = 'TX.VAL.MRCH.CD.WT' AND country_name = 'Indonesia', indicator_value, 0)) merch_import_us_dollar
               FROM `patents-public-data.worldbank_wdi.wdi_2016`
               GROUP BY year)
        WHERE year >= 2006 AND year < 2016
        """
result = client.query(query).to_dataframe()
result.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,merch_export_us_dollar,merch_import_us_dollar,difference
0,2006,8.064969e+10,1.035270e+11,-2.287731e+10
1,2007,9.310060e+10,1.180130e+11,-2.491240e+10
2,2008,1.275385e+11,1.396060e+11,-1.206754e+10
3,2009,9.378593e+10,1.196460e+11,-2.586007e+10
4,2010,1.356633e+11,1.577791e+11,-2.211582e+10
5,2011,1.774356e+11,2.034966e+11,-2.606107e+10
6,2012,1.916909e+11,1.900318e+11,1.659069e+09
7,2013,1.866286e+11,1.825518e+11,4.076877e+09
8,2014,1.781788e+11,1.762927e+11,1.886136e+09
9,2015,1.426948e+11,1.502824e+11,-7.587600e+09


The GDP of a country tends to increase when the total value of goods and services that domestic producers sell to foreign countries exceeds the total value of foreign goods and services that domestic consumers buy. When this situation occurs, a country is said to have a trade surplus. If the opposite situation occurs, if the amount that domestic consumers spend on foreign products is greater than the total sum of what domestic producers are able to sell to foreign consumers—it is called a trade deficit. In this situation, the GDP of a country tends to decrease. From 2006 to 2015 (10 years) Indonesia's balance of Trade mostly in trade deficit situation,